In [43]:
from scipy.stats import mannwhitneyu, wilcoxon
import json
from statannot import add_stat_annotation
import seaborn as sns
import pandas as pd
from functools import reduce

In [44]:
file = "/workspaces/RAG_secure_code_generation/plots_rq2.csv"

In [45]:
df = pd.read_csv(file)
df.head()

,experiment,dataset,avg_accuracy,top_1_acc,top_1_acc_diff,top_1_acc_improvement,top_3_acc,top_3_acc_diff,top_3_acc_improvement,top_5_acc,top_5_acc_diff,top_5_acc_improvement
0,gpt-4-0125-preview_0.5_rag_few_shot_3,anthropic-claude-3-haiku_0.5_zero_shot,0.765,0.838,0.146,0.073,0.852,0.117,0.088,0.833,0.125,0.068
1,gpt-4-0125-preview_0.5_rag_few_shot_3,anthropic-claude-3-haiku_0.5_few_shot_3,0.765,0.923,0.061,0.158,0.887,0.082,0.122,0.877,0.081,0.112
2,gpt-4-0125-preview_0.5_rag_few_shot_3,anthropic-claude-3-haiku_0.5_few_shot_1,0.765,0.810,0.174,0.046,0.830,0.139,0.065,0.854,0.104,0.089
3,gpt-4-0125-preview_0.5_rag_few_shot_3,anthropic-claude-3-haiku_0.5_few_shot_5,0.765,0.968,0.016,0.204,0.901,0.069,0.136,0.866,0.092,0.101
4,gpt-4-0125-preview_0.5_rag_few_shot_3,anthropic-claude-3-haiku_1.0_zero_shot_5,0.765,0.835,0.148,0.071,0.854,0.115,0.089,0.845,0.113,0.080


In [46]:
top = 1
rq2_df_top = df[["experiment", "dataset",f"top_{top}_acc", f"top_{top}_acc_diff"]]
rq3_df_top = df[["experiment", "dataset", "avg_accuracy",f"top_{top}_acc"]]

In [47]:
rq2_df_top.head()

,experiment,dataset,top_1_acc,top_1_acc_diff
0,gpt-4-0125-preview_0.5_rag_few_shot_3,anthropic-claude-3-haiku_0.5_zero_shot,0.838,0.146
1,gpt-4-0125-preview_0.5_rag_few_shot_3,anthropic-claude-3-haiku_0.5_few_shot_3,0.923,0.061
2,gpt-4-0125-preview_0.5_rag_few_shot_3,anthropic-claude-3-haiku_0.5_few_shot_1,0.810,0.174
3,gpt-4-0125-preview_0.5_rag_few_shot_3,anthropic-claude-3-haiku_0.5_few_shot_5,0.968,0.016
4,gpt-4-0125-preview_0.5_rag_few_shot_3,anthropic-claude-3-haiku_1.0_zero_shot_5,0.835,0.148


In [48]:
rq2_df_top["optimal_top"] = df[f"top_{top}_acc"] + df[f"top_{top}_acc_diff"]

/tmp/ipykernel_256718/4288659812.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rq2_df_top["optimal_top"] = df[f"top_{top}_acc"] + df[f"top_{top}_acc_diff"]


In [49]:
rq2_df_top.head()

,experiment,dataset,top_1_acc,top_1_acc_diff,optimal_top
0,gpt-4-0125-preview_0.5_rag_few_shot_3,anthropic-claude-3-haiku_0.5_zero_shot,0.838,0.146,0.984
1,gpt-4-0125-preview_0.5_rag_few_shot_3,anthropic-claude-3-haiku_0.5_few_shot_3,0.923,0.061,0.984
2,gpt-4-0125-preview_0.5_rag_few_shot_3,anthropic-claude-3-haiku_0.5_few_shot_1,0.810,0.174,0.984
3,gpt-4-0125-preview_0.5_rag_few_shot_3,anthropic-claude-3-haiku_0.5_few_shot_5,0.968,0.016,0.984
4,gpt-4-0125-preview_0.5_rag_few_shot_3,anthropic-claude-3-haiku_1.0_zero_shot_5,0.835,0.148,0.983


In [50]:
rq2_df_top = rq2_df_top.groupby(["experiment"]).agg({"top_1_acc": list, "optimal_top": list})
rq2_df_top.head()

,top_1_acc,optimal_top
experiment,,
anthropic-claude-3-opus_0.0_few_shot_1,"[0.857, 0.959, 0.892, 0.953, 0.841, 0.95, 0.90...","[0.983, 0.983, 0.984, 0.983, 0.983, 0.983, 0.9..."
anthropic-claude-3-opus_0.0_few_shot_3,"[0.958, 0.987, 0.988, 0.995, 0.975, 0.974, 0.9...","[0.995, 0.995, 0.995, 0.995, 0.995, 0.995, 0.9..."
anthropic-claude-3-opus_0.0_few_shot_5,"[0.857, 0.949, 0.92, 0.949, 0.92, 0.918, 0.895...","[0.983, 0.983, 0.9830000000000001, 0.983, 0.98..."
anthropic-claude-3-opus_0.0_rag_0,"[0.948, 0.947, 0.952, 0.954, 0.942, 0.953, 0.9...","[0.977, 0.978, 0.977, 0.977, 0.978, 0.977, 0.9..."
anthropic-claude-3-opus_0.0_rag_few_shot_1,"[0.636, 0.894, 0.658, 0.824, 0.68, 0.9, 0.813,...","[0.9750000000000001, 0.975, 0.9750000000000001..."


In [51]:
rq2_df_top["pvalue"] = rq2_df_top.apply(lambda x: mannwhitneyu(x["top_1_acc"], x["optimal_top"]).pvalue, axis=1)
rq2_df_top.head()

,top_1_acc,optimal_top,pvalue
experiment,,,
anthropic-claude-3-opus_0.0_few_shot_1,"[0.857, 0.959, 0.892, 0.953, 0.841, 0.95, 0.90...","[0.983, 0.983, 0.984, 0.983, 0.983, 0.983, 0.9...",3.155965e-22
anthropic-claude-3-opus_0.0_few_shot_3,"[0.958, 0.987, 0.988, 0.995, 0.975, 0.974, 0.9...","[0.995, 0.995, 0.995, 0.995, 0.995, 0.995, 0.9...",2.049664e-23
anthropic-claude-3-opus_0.0_few_shot_5,"[0.857, 0.949, 0.92, 0.949, 0.92, 0.918, 0.895...","[0.983, 0.983, 0.9830000000000001, 0.983, 0.98...",1.426167e-22
anthropic-claude-3-opus_0.0_rag_0,"[0.948, 0.947, 0.952, 0.954, 0.942, 0.953, 0.9...","[0.977, 0.978, 0.977, 0.977, 0.978, 0.977, 0.9...",4.940351e-21
anthropic-claude-3-opus_0.0_rag_few_shot_1,"[0.636, 0.894, 0.658, 0.824, 0.68, 0.9, 0.813,...","[0.9750000000000001, 0.975, 0.9750000000000001...",1.109716e-22


In [52]:
len(rq2_df_top)


136

In [53]:
#print the len where pvalue is less than 0.05
print(len(rq2_df_top[rq2_df_top["pvalue"] < 0.05]))

135


In [54]:
df.experiment.unique()

array(['gpt-4-0125-preview_0.5_rag_few_shot_3',
       'gpt-4-0125-preview_0.0_rag_few_shot_3',
       'gpt-4-0125-preview_1.0_rag_few_shot_3',
       'gpt-4-0125-preview_0.5_rag_few_shot_1',
       'gpt-4-0125-preview_0.0_rag_few_shot_1',
       'gpt-4-0125-preview_1.0_rag_few_shot_1',
       'gpt-4-0125-preview_0.5_rag_few_shot_5',
       'gpt-4-0125-preview_0.0_rag_few_shot_5',
       'gpt-4-0125-preview_1.0_rag_few_shot_5',
       'gpt-4-0125-preview_0.5_zero_shot_0',
       'gpt-4-0125-preview_0.0_zero_shot_0',
       'gpt-4-0125-preview_1.0_zero_shot_0',
       'gpt-4-0125-preview_0.5_few_shot_3',
       'gpt-4-0125-preview_0.0_few_shot_3',
       'gpt-4-0125-preview_1.0_few_shot_3',
       'gpt-4-0125-preview_0.5_few_shot_1',
       'gpt-4-0125-preview_0.0_few_shot_1',
       'gpt-4-0125-preview_1.0_few_shot_1',
       'gpt-4-0125-preview_0.5_few_shot_5',
       'gpt-4-0125-preview_0.0_few_shot_5',
       'gpt-4-0125-preview_1.0_few_shot_5',
       'gpt-4-0125-preview_0.5_rag_0'

In [55]:
#count nan values
print(df.isna().sum())

experiment               0
dataset                  0
avg_accuracy             0
top_1_acc                0
top_1_acc_diff           0
top_1_acc_improvement    0
top_3_acc                0
top_3_acc_diff           0
top_3_acc_improvement    0
top_5_acc                0
top_5_acc_diff           0
top_5_acc_improvement    0
dtype: int64


In [56]:
top_k_accuracies = df[f"top_{1}_acc"].values.tolist()
avg_accuracies = df[f"avg_accuracy"].values.tolist()
top_k_accuracies, len(top_k_accuracies)

([0.838,
  0.923,
  0.81,
  0.968,
  0.835,
  0.926,
  0.877,
  0.927,
  0.818,
  0.844,
  0.833,
  0.856,
  0.856,
  0.869,
  0.889,
  0.852,
  0.856,
  0.82,
  0.881,
  0.847,
  0.823,
  0.937,
  0.898,
  0.872,
  0.855,
  0.854,
  0.834,
  0.852,
  0.909,
  0.854,
  0.875,
  0.852,
  0.872,
  0.853,
  0.94,
  0.854,
  0.849,
  0.848,
  0.851,
  0.922,
  0.866,
  0.932,
  0.856,
  0.826,
  0.889,
  0.82,
  0.879,
  0.79,
  0.822,
  0.765,
  0.843,
  0.868,
  0.884,
  0.859,
  0.897,
  0.861,
  0.87,
  0.952,
  0.863,
  0.894,
  0.834,
  0.854,
  0.854,
  0.893,
  0.839,
  0.907,
  0.879,
  0.889,
  0.91,
  0.902,
  0.87,
  0.88,
  0.868,
  0.883,
  0.881,
  0.902,
  0.872,
  0.875,
  0.881,
  0.872,
  0.91,
  0.873,
  0.841,
  0.948,
  0.942,
  0.94,
  0.883,
  0.916,
  0.9,
  0.898,
  0.939,
  0.906,
  0.935,
  0.897,
  0.897,
  0.926,
  0.929,
  0.907,
  0.858,
  0.864,
  0.894,
  0.868,
  0.878,
  0.842,
  0.889,
  0.881,
  0.91,
  0.874,
  0.879,
  0.831,
  0.854,
  0.855,
  0.89

In [57]:
avg_accuracies, len(avg_accuracies)

([0.765,
  0.765,
  0.765,
  0.765,
  0.765,
  0.765,
  0.765,
  0.765,
  0.765,
  0.765,
  0.765,
  0.765,
  0.765,
  0.765,
  0.765,
  0.765,
  0.765,
  0.765,
  0.765,
  0.765,
  0.765,
  0.765,
  0.765,
  0.765,
  0.765,
  0.765,
  0.765,
  0.765,
  0.765,
  0.765,
  0.765,
  0.765,
  0.765,
  0.765,
  0.765,
  0.765,
  0.765,
  0.765,
  0.765,
  0.765,
  0.765,
  0.765,
  0.765,
  0.765,
  0.765,
  0.765,
  0.765,
  0.765,
  0.765,
  0.765,
  0.765,
  0.765,
  0.765,
  0.765,
  0.765,
  0.765,
  0.765,
  0.765,
  0.765,
  0.765,
  0.765,
  0.765,
  0.814,
  0.814,
  0.814,
  0.814,
  0.814,
  0.814,
  0.814,
  0.814,
  0.814,
  0.814,
  0.814,
  0.814,
  0.814,
  0.814,
  0.814,
  0.814,
  0.814,
  0.814,
  0.814,
  0.814,
  0.814,
  0.814,
  0.814,
  0.814,
  0.814,
  0.814,
  0.814,
  0.814,
  0.814,
  0.814,
  0.814,
  0.814,
  0.814,
  0.814,
  0.814,
  0.814,
  0.814,
  0.814,
  0.814,
  0.814,
  0.814,
  0.814,
  0.814,
  0.814,
  0.814,
  0.814,
  0.814,
  0.814,
  0.814,
 

In [62]:
stat = wilcoxon(top_k_accuracies, avg_accuracies)


In [63]:
stat

WilcoxonResult(statistic=245556.0, pvalue=0.0)

In [65]:
def a12_unpaired(lst1, lst2):
    more = same = 0.0
    for x in lst1:
        for y in lst2:
            if x == y: same +=1
            elif x > y: more +=1
    return (more + 0.5*same)/(len(lst1)*len(lst2))

def interpret_effect_size(effect_size_score):
    if effect_size_score < 0.147:
        return "negligible"
    elif effect_size_score < 0.33:
        return "small"
    elif effect_size_score < 0.474:
        return "medium"
    else:
       return "large"

In [66]:
a12 = a12_unpaired(top_k_accuracies, avg_accuracies)

effect_size_score = 2*abs(a12 -0.5)
effect_size = interpret_effect_size(effect_size_score)

In [68]:
effect_size

'large'